## **EDIT THE BUCKET NAME** ##

In [1]:
account_id = "199131085527" # edit this to be the s3 bucket name for your model-data-bucket. It should be model-data-bucket-<accountid>
neptune_cluster_url = "neptunenotebookstack-neptunecdkcluster6a0221d7-ohmwwaexm9y7.cluster-cxsturxlbxux.us-east-1.neptune.amazonaws.com"
region = "us-east-1"

# Libraries and global variable definitions

In [2]:
import awswrangler as wr
import pandas as pd
import urllib.parse
from datetime import datetime, timedelta
from rdflib import RDF, RDFS, OWL, Namespace, Graph,URIRef, Literal
from rdflib.namespace import XSD
import boto3

s3 = boto3.resource("s3")
turtleFileDirPath= "asset-input-data/turtle-files/"
csv_name = "SITE_01.csv"
bucket_name = f"model-data-bucket-{account_id}"
assets_csv_path= f"s3://{bucket_name}/{csv_name}"
turtle_s3_path = f"s3://{bucket_name}/{turtleFileDirPath}"
load_neptune_iam_role = f"arn:aws:iam::{account_id}:role/neptune-load-from-s3-{account_id}"

# Functions to create brick turtle file

In [3]:
def determineAssetType(slotpath_text):
    if "RTU" in slotpath_text.upper():
        return ('RTU')
    elif 'GBL' or 'globalLogicOffice'.upper or 'oaConditions'.upper in slotpath_text.upper():
        return ('SiteLevelAsset')
    else:
        return ('unkown')

def buildBrickTurtleFile(turtleFileDirPath,assets_csv_path):
    
    print(turtleFileDirPath)
    print(assets_csv_path)
    file_name = assets_csv_path.split("/")[-1] 
    site_name = (file_name.split(".")[0]).lower()
    
    building_namespace = "http://amazon.bms.com/building-"+ site_name + "#"
    g = Graph()
    BMS = Namespace(building_namespace)
    g.bind("bms", BMS)

    BRICK = Namespace("https://brickschema.org/schema/Brick#")
    g.bind("brick", BRICK)

    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    g.bind("rdfs", RDFS)
    
    #read csv file 
    try:
        assets_csv = wr.s3.read_csv(path=assets_csv_path)
    except:
        print("no data file located")
        
    #get values from each row and get url valid string 
    endoded_site_name = urllib.parse.quote_plus(site_name)   
    
    if(site_name):
        g.add(((BMS[URIRef(endoded_site_name)]), RDF.type, BRICK.Building))
        g.add(((BMS[URIRef(endoded_site_name)]), RDFS.label, Literal(site_name))) 

    #parse the CSV file and create turtle file 
    for asset_row in assets_csv.itertuples():

        asset_name = (asset_row[2]).strip()
        endoded_asset_name = urllib.parse.quote_plus(asset_name)

        asset_timeSeries_identify = (asset_row[3]).strip()

        asset_type = (asset_row[4]).strip()
        endoded_asset_type = urllib.parse.quote_plus(asset_type)

        point_name = (asset_row[5]).strip()
        endoded_point_name = urllib.parse.quote_plus(point_name)

        point_timeSeries_identify = (asset_row[6]).strip()

        if(asset_name):
            brickAssetType= determineAssetType(asset_type)
            g.add(((BMS[URIRef(endoded_asset_name)]), RDF.type, BRICK[(brickAssetType)]))
            g.add((BMS[URIRef(endoded_asset_name)], BRICK.hasLocation, (BMS[URIRef(endoded_site_name)])))
            g.add((BMS[URIRef(endoded_asset_name)], BRICK.timeseries, (Literal(asset_timeSeries_identify))))
            g.add(((BMS[URIRef(endoded_asset_name)]), RDFS.label, Literal(asset_name))) 

        if(point_name):
            g.add(((BMS[URIRef((endoded_site_name))]), BRICK.hasPoint, (BMS[URIRef(endoded_point_name)])))
            g.add(((BMS[URIRef((endoded_asset_name))]), BRICK.hasPoint, (BMS[URIRef(endoded_point_name)])))
            g.add(((BMS[URIRef(endoded_point_name)]), RDF.type, (BRICK[(endoded_point_name)])))
            g.add((BMS[URIRef(endoded_point_name)], BRICK.timeseries, (Literal(point_timeSeries_identify))))
            g.add(((BMS[URIRef(endoded_point_name)]), RDFS.label, Literal(point_name))) 
    #write the ttl to s3 
    file_name = site_name +"_amazon_bms_demo.ttl"
    s3_path = turtleFileDirPath + file_name
    s3.Bucket(bucket_name).put_object(Key=s3_path, Body=(g.serialize(format="turtle")))
    return(building_namespace)

In [4]:
result= buildBrickTurtleFile(turtleFileDirPath,assets_csv_path)
print(result)

asset-input-data/turtle-files/
s3://model-data-bucket-199131085527/SITE_01.csv
http://amazon.bms.com/building-site_01#


In [5]:
turtle_s3_path

's3://model-data-bucket-199131085527/asset-input-data/turtle-files/'

# Upload turtle file to neptune

In [15]:
import requests
import json

#load team nodes from s3 
neptuneLoadUrl = f'https://{neptune_cluster_url}:8182/loader'
payload = {"source":turtle_s3_path, "format":"turtle", "iamRoleArn":load_neptune_iam_role, "region":region, "failOnError":"FALSE", "parallelism":"MEDIUM", "updateSingleCardinalityProperties":"TRUE", "queueRequest":"TRUE"}
res = requests.post(neptuneLoadUrl, data=payload)
print(res.text)

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "82a08cfa-cabf-4c41-bf65-e41c164af1e3"
    }
}


In [18]:
res = requests.get(neptuneLoadUrl+'/' + json.loads(res.text)["payload"]["loadId"])

res.json()

{'status': '200 OK',
 'payload': {'feedCount': [{'LOAD_COMPLETED': 1}],
  'overallStatus': {'fullUri': 's3://model-data-bucket-199131085527/asset-input-data/turtle-files/',
   'runNumber': 3,
   'retryNumber': 0,
   'status': 'LOAD_COMPLETED',
   'totalTimeSpent': 3,
   'startTime': 1682467625,
   'totalRecords': 51,
   'totalDuplicates': 51,
   'parsingErrors': 0,
   'datatypeMismatchErrors': 0,
   'insertErrors': 0}}}